In [1]:
using Oscar;
using Combinatorics;
pm = Polymake;
include("InitialIdealsRegularSubdivisions.jl");

  ___   ___   ___    _    ____
 / _ \ / __\ / __\  / \  |  _ \  | Combining and extending ANTIC, GAP,
| |_| |\__ \| |__  / ^ \ |  ´ /  | Polymake and Singular
 \___/ \___/ \___//_/ \_\|_|\_\  | Type "?Oscar" for more information
o--------o-----o-----o--------o  | Documentation: https://docs.oscar-system.org
  S Y M B O L I C   T O O L S    | Version 1.6.0


In [2]:
Plu24, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4], 2));
I24 = grassmann_pluecker_ideal(Plu24, 2, 4);
Delta24 = hypersimplex(2,4);
Delta24_polymake = pm.polytope.hypersimplex(2,4);
Sec24 = polyhedral_fan(pm.fan.secondary_fan(Delta24_polymake));
F = Omega_fan(I24, Delta24, Sec24)
is_complete(F)

LoadError: MethodError: no method matching zero(::Type{Any})
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

[0mClosest candidates are:
[0m  zero(::Type{Union{Missing, T}}) where T
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:105[24m[39m
[0m  zero([91m::Type{Union{}}[39m, Any...)
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:315[24m[39m
[0m  zero([91m::Type{RootSpaceElem}[39m, [91m::RootSystem[39m)
[0m[90m   @[39m [36mOscar[39m [90m~/.julia/packages/Oscar/eTXAp/src/LieTheory/[39m[90m[4mRootSystem.jl:799[24m[39m
[0m  ...


In [6]:
Sec = Sec24
if typeof(Sec) == PolyhedralFan{QQFieldElem}
    rays_Delta = rays_modulo_lineality(Sec)[1];
    cones_Delta = cones(Sec);
else
    rays_Delta, cones_Delta = Sec
end

n_cones_Delta, n_rays_Delta = size(cones_Delta)

    reps = [sum([rays_Delta[j] for j in 1:n_rays_Delta if cones_Delta[i,j]]) 
                               for i in 1:n_cones_Delta];

(7, 3)

In [ ]:
Plu25, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4,5], 2));
I25 = grassmann_pluecker_ideal(Plu25, 2, 5);
Delta25 = hypersimplex(2,5);

Delta25_polymake = Polymake.polytope.hypersimplex(2,5);
Sec25 = polyhedral_fan(Polymake.fan.secondary_fan(Delta25_polymake));
Omega25 = Omega_fan(I25, Delta25, Sec25, true);
Omega25[2]

In [ ]:
I26 = grassmann_pluecker_ideal(2, 6);
Sec26 = load("secondary_fan_D(2,6).fan")
R = reshape(Sec26.pm_fan.RAYS, 1235, :)
GR = Bool[]
for r in eachrow(R)
#     w = sum(rays_modulo_lineality(c)[1])
    w = QQ.(lcm(denominator.(r))*r)
    inI = initial(I26, tropical_semiring_map(QQ), w)
    Iw = ideal_w(I26, w)
    push!(GR, Iw == inI)
end
sum(GR)

In [ ]:
MC = reshape(Sec26.pm_fan.MAXIMAL_CONES, 194160, :)
c = 0
for i in 1 : 194160
    if prod(MC[i, :] .<= GR) == 1
        w = sum(R[j, :] for j in MC[i, :].s)
        w = Int.(lcm(denominator.(w))*w)
        inI = initial(I26, tropical_semiring_map(QQ), w)
        Iw = ideal_w(I26, w)
        c += inI == Iw
    end
end
c